## Evaluating the visual-based diffusion policy on a hard episode, 250 times across 5 random seeds

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# setup vulkan
!mkdir -p /usr/share/vulkan/icd.d
!wget -q https://raw.githubusercontent.com/haosulab/ManiSkill/main/docker/nvidia_icd.json
!wget -q https://raw.githubusercontent.com/haosulab/ManiSkill/main/docker/10_nvidia.json
!mv nvidia_icd.json /usr/share/vulkan/icd.d
!mv 10_nvidia.json /usr/share/glvnd/egl_vendor.d/10_nvidia.json
!apt-get install -y --no-install-recommends libvulkan-dev
# dependencies
!pip install --upgrade mani_skill tyro

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libvulkan-dev is already the newest version (1.3.204.1-2).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [3]:
# Import required packages
import gymnasium as gym
from tqdm.notebook import tqdm
import numpy as np
import mani_skill.envs
import matplotlib.pyplot as plt

In [4]:
import sys
sys.path.append('/content/drive/MyDrive/diffusionpolicy')

In [5]:
from diffusion_policy.evaluate import evaluate
from diffusion_policy.make_env import make_eval_envs
from train_rgbd_2b import Agent, Args
import torch
import numpy as np

In [6]:
!ls /content/drive/MyDrive/diffusionpolicy/runs/diffusion_policy-PushT-v1-rgbd-2b-00_rl_demos-/checkpoints

best_eval_success_once.pt


In [17]:
ckpt_path = "/content/drive/MyDrive/diffusionpolicy/runs/diffusion_policy-PushT-v1-rgbd-2b-00_rl_demos-/checkpoints/best_eval_success_once.pt"
ckpt = torch.load(ckpt_path, map_location='cuda')

In [18]:
args = Args(
    env_id="PushT-v1",
    obs_mode="rgbd",
    control_mode="pd_ee_delta_pos",
    sim_backend="physx_cuda",
    num_eval_envs=10,
    max_episode_steps=100,
    obs_horizon=2
)

In [19]:
env_kwargs = dict(
      control_mode=args.control_mode,
      reward_mode="sparse",
      obs_mode=args.obs_mode,
      render_mode="rgb_array",
      human_render_camera_configs=dict(shader_pack="default")
)

other_kwargs = dict(obs_horizon=args.obs_horizon)

In [21]:
from mani_skill.utils.wrappers.flatten import FlattenRGBDObservationWrapper

envs = make_eval_envs(
    args.env_id,
    args.num_eval_envs,
    args.sim_backend,
    env_kwargs,
    other_kwargs,
    video_dir=None,
    wrappers=[FlattenRGBDObservationWrapper],
)

In [22]:
agent = Agent(envs, args)
agent.load_state_dict(ckpt["ema_agent"])
agent.eval()
agent = agent.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

number of parameters: 6.07M


In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
agent = agent.to(device)
agent.device = device

In [25]:
results = []
for seed in range(5):
    np.random.seed(seed)
    torch.manual_seed(seed)
    envs.reset(seed=seed)
    metrics = evaluate(
        n=250,
        agent=agent,
        eval_envs=envs,
        device=agent.device if hasattr(agent, 'device') else torch.device("cpu"),
        sim_backend=args.sim_backend,
        progress_bar=True
    )
    results.append(metrics)
    print(f"Seed {seed}: success_once mean={np.mean(metrics['success_once']):.4f}, "
          f"success_at_end mean={np.mean(metrics['success_at_end']):.4f}")


100%|██████████| 250/250 [07:39<00:00,  1.84s/it]


Seed 0: success_once mean=0.0000, success_at_end mean=0.0000



100%|██████████| 250/250 [07:41<00:00,  1.85s/it]


Seed 1: success_once mean=0.0000, success_at_end mean=0.0000



100%|██████████| 250/250 [07:32<00:00,  1.81s/it]


Seed 2: success_once mean=0.0000, success_at_end mean=0.0000



100%|██████████| 250/250 [07:33<00:00,  1.82s/it]


Seed 3: success_once mean=0.0000, success_at_end mean=0.0000



100%|██████████| 250/250 [07:36<00:00,  1.82s/it]

Seed 4: success_once mean=0.0000, success_at_end mean=0.0000
